In [1]:
import pandas as pd
from kinconnect_api.config import RAW_DATA_DIR, load_dotenv

load_dotenv()
# fake_profile_data = pd.read_csv(f"{RAW_DATA_DIR}/fake_profile_data.csv")
# fake_profile_data.head()
fake_profile_data = pd.read_csv(f"{RAW_DATA_DIR}/fake_profile_data_2.tsv", sep="\t")
fake_profile_data.head()

2024-06-14 17:48:45.371 | INFO     | kinconnect_api.config:<module>:15 - PROJ_ROOT path is: /Users/nehiljain/code/kinconnect/kinconnect_api


,"What are your interests? (ie technical topic, coding language, business problem).","If you have a project idea, describe your idea . Please include whether what sector it is in, and what business problem it is solving and for whom. If you don’t have a project, skip this question.","What is your strongest functional role (such as developer, UX, business, product)? Please share one or two things about your experience in role your experience, for example a success, companies you worked for, how many years experience, a challenging project, etc.",Career path from Linkedin,Are you interested in meeting people with a specific skill set (either one that you lack or one that you already have but want to clone yourself to speed up building). What is the skills sets that you are looking to meet?,Past Projects Portfolio
0,I'm interested in Rags and AI LLMs,NaN,"Back end developer, 10 years, build and deploy...",Senior Software EngineerSenior Software Engine...,"Product, frontend,",### Project 1: **Netflix Recommendation Engine...
1,I'm interested in using AI to solve climate ch...,How can AI use to regulate office temperature ...,Product manager. I worked as a product manager...,NaN,"Front end developer, Back end developer",### Project 1: **Uber Driver App Redesign** *...
2,I'm interested in using AI to make building co...,Using AI to make building construction and dev...,Business Strategy. Business Development. Marke...,NaN,"Front end developer, Back end developer",### Project 1: **Market Expansion Strategy** ...
3,I'm interested in using AI to solve the loneli...,Match making,Frontend engineer. I worked at building the U...,NaN,"Business strategy, Back end developer, product",### Project 1: **Real-Time Incident Reporting ...
4,I'm interested in using AI to create more empathy,How can AI be used to make people think more a...,Backend engineer. I have worked on databases f...,Micro Quant Fund · Full-timeMicro Quant Fund ·...,Product managers. Strategic business. Maybe so...,### Project 1: **Slack Data Synchronization Sy...


In [2]:
profile_list = []

for index, row in fake_profile_data.iterrows():
    profile_dict = {'questions_and_answers': []}
    for column in fake_profile_data.columns:
        qa_pair = {"question": column, "answer": row[column]}
        profile_dict['questions_and_answers'].append(qa_pair)
    profile_list.append(profile_dict)

profile_details_list = []
for index, row in fake_profile_data.iterrows():
    profile_dict = {'questions_and_answers': []}
    counter = 0
    for column in fake_profile_data.columns:
        qa_pair = {"question": column, "answer": row[column]}
        profile_dict['questions_and_answers'].append(qa_pair)
        print(qa_pair)
        if counter >= 2:
            break
        counter += 1
    profile_details_list.append(profile_dict)

# Convert each profile's questions and answers into a formatted string and store in a list
formatted_profiles = [f'\n\n'.join(f"## {pair['question']}: \n {pair['answer']}" for pair in profile['questions_and_answers']) for profile in profile_list]
formatted_profile_details = [f'\n\n'.join(f"## {pair['question']}: \n {pair['answer']}" for pair in profile['questions_and_answers']) for profile in profile_details_list]
len(formatted_profiles)


{'question': 'What are your interests?  (ie technical topic, coding language, business problem).', 'answer': "I'm interested in Rags and AI LLMs"}
{'question': 'If you have a project idea, describe your idea . Please include whether what sector it is in, and what business problem it is solving and for whom. If you don’t have a project, skip this question.', 'answer': nan}
{'question': 'What is your strongest functional role (such as developer, UX, business, product)? Please share one or two things about your experience in role your experience, for example a success, companies you worked for, how many years experience, a challenging project, etc.', 'answer': 'Back end developer, 10 years, build and deployed backend databases for Netflix including adding AI functionality to Netflix recommendation engines. '}
{'question': 'What are your interests?  (ie technical topic, coding language, business problem).', 'answer': "I'm interested in using AI to solve climate change"}
{'question': 'If yo

11

In [3]:
print(formatted_profile_details[0])

## What are your interests?  (ie technical topic, coding language, business problem).: 
 I'm interested in Rags and AI LLMs

## If you have a project idea, describe your idea . Please include whether what sector it is in, and what business problem it is solving and for whom. If you don’t have a project, skip this question.: 
 nan

## What is your strongest functional role (such as developer, UX, business, product)? Please share one or two things about your experience in role your experience, for example a success, companies you worked for, how many years experience, a challenging project, etc.: 
 Back end developer, 10 years, build and deployed backend databases for Netflix including adding AI functionality to Netflix recommendation engines. 


In [4]:
print(formatted_profiles[1])

## What are your interests?  (ie technical topic, coding language, business problem).: 
 I'm interested in using AI to solve climate change

## If you have a project idea, describe your idea . Please include whether what sector it is in, and what business problem it is solving and for whom. If you don’t have a project, skip this question.: 
 How can AI use to regulate office temperature more efficiently depending on where people are in a building

## What is your strongest functional role (such as developer, UX, business, product)? Please share one or two things about your experience in role your experience, for example a success, companies you worked for, how many years experience, a challenging project, etc.: 
 Product manager. I worked as a product manager at Uber, helping to build the mobile app for drivers. 

## Career path from Linkedin: 
 nan

## Are you interested in meeting people with a specific skill set (either one that you lack or one that you already have but want to clon

In [5]:
from pymongo import MongoClient
from pydantic import BaseModel, Field
from typing import List, Dict

class CareerEntry(BaseModel):
    company: str = Field(..., title="Company they worked at")
    title: str = Field(..., title="Title of the role they held")
    description: str = Field(..., title="Description of the role they held")
    start_date: str = Field(..., title="Start date of the role")
    end_date: str = Field(..., title="End date of the role")


class ProjectEntry(BaseModel):
    title: str = Field(..., title="Title of the project")
    description: str = Field(..., title="Description of the project")




In [6]:

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.chat_models.base import BaseChatModel
from textwrap import dedent
import logging
from pathlib import Path
import pandas as pd
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List, Dict, Callable, Union
import os
from tqdm import tqdm
from datetime import datetime
# Setup logging as per the new configuration

In [7]:
career_extraction_prompt_str = dedent("""
You are a data generation app which creates profiles for participants at a hackathon. 
Given their form submission which asks them a few questions. Create a profile. 
Find all the relevant details about their career.

<form_submission>                                   
{submission}
</form_submission>
""")

In [8]:
from langchain_fireworks import ChatFireworks
load_dotenv()

def get_career(profile):
    prompt_str = dedent("""
You are a data generation app which creates profiles for participants at a hackathon. 
Given their form submission which asks them a few questions. Create a profile. 
Find all the relevant details about their career.

<form_submission>                                   
{submission}
</form_submission>
""")
    mixtral_llm = ChatFireworks(
        model="accounts/fireworks/models/mistral-7b-instruct-v3")

    career_llm = mixtral_llm.with_structured_output(CareerEntry)

    prompt = ChatPromptTemplate.from_template(prompt_str)
    chain = prompt | career_llm
    return chain.invoke({"submission": profile})


career_profiles = [get_career(unstructured_profile) for unstructured_profile in formatted_profiles]


In [9]:
career_profiles[0]


{'company': 'Netflix',
 'title': 'Senior Software Engineer',
 'description': 'Led the development and deployment of an advanced recommendation engine for Netflix. This project aimed to enhance the accuracy and personalization of content recommendations for users by integrating machine learning algorithms. The system utilized user behavior data, viewing history, and ratings to predict and suggest content that matched user preferences. The project included a real-time processing pipeline to ensure recommendations were updated dynamically as user interactions occurred.',
 'start_date': 'Jun 2018',
 'end_date': 'Present'}

In [10]:
from langchain_fireworks import ChatFireworks
load_dotenv()

def get_project(profile):
    prompt_str = dedent("""
You are a data generation app which creates profiles for participants at a hackathon. 
Given their form submission which asks them a few questions. Create a profile. 
Find all the relevant details about their past projects and portfolio.

<form_submission>                                   
{submission}
</form_submission>
""")
    mixtral_llm = ChatFireworks(
        model="accounts/fireworks/models/mistral-7b-instruct-v3")

    career_llm = mixtral_llm.with_structured_output(ProjectEntry)

    prompt = ChatPromptTemplate.from_template(prompt_str)
    chain = prompt | career_llm
    return chain.invoke({"submission": profile})


project_profiles = [get_project(unstructured_profile) for unstructured_profile in formatted_profiles]


In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from textwrap import dedent
from langchain_core.pydantic_v1 import BaseModel, Field

load_dotenv()

def get_profile_details(profile):
    prompt_str = dedent("""
You are a data generation app which creates profiles for participants at a hackathon. 
Given their form submission which asks them a few questions. Create a profile. 
Find all the relevant details about their profile. Find their skills, accolades, honors and current interests.
Generate a name for the person. Make the name diverse and real representation of people working in the silicon valley with the relevant description of their form submission.
                        
<form_submission>                                   
{submission}
</form_submission>
""")
    # Define a Pydantic model for the profile
    class ProfileDetails(BaseModel):
        honors: List[str] = Field(None, title="Honors, Awards and recognition they have received in life")
        interests: List[str] = Field(..., title="Interests and current focus of theirs the work or the event")
        skills: List[str] = Field(..., title="Skills they have")
        name: str = Field(..., title="Name of the person")
        elevator_pitch: str = Field(..., title="Elevator pitch for the person for the event")
        
    gpt35_llm =  ChatOpenAI(model="gpt-3.5-turbo")

    career_llm = gpt35_llm.with_structured_output(ProfileDetails)

    prompt = ChatPromptTemplate.from_template(prompt_str)
    chain = prompt | career_llm
    return chain.invoke({"submission": profile})


profile_details = [get_profile_details(unstructured_profile) for unstructured_profile in formatted_profile_details]


In [12]:
print(career_profiles[0])
print(project_profiles[0])
print(profile_details[0])

{'company': 'Netflix', 'title': 'Senior Software Engineer', 'description': 'Led the development and deployment of an advanced recommendation engine for Netflix. This project aimed to enhance the accuracy and personalization of content recommendations for users by integrating machine learning algorithms. The system utilized user behavior data, viewing history, and ratings to predict and suggest content that matched user preferences. The project included a real-time processing pipeline to ensure recommendations were updated dynamically as user interactions occurred.', 'start_date': 'Jun 2018', 'end_date': 'Present'}
{'title': 'Netflix Recommendation Engine Enhancement', 'description': 'Led the development and deployment of an advanced recommendation engine for Netflix. This project aimed to enhance the accuracy and personalization of content recommendations for users by integrating machine learning algorithms. The system utilized user behavior data, viewing history, and ratings to predic

In [13]:
from kinconnect_api.config import MONGO_CONNECTION_STRING
# Connect to MongoDB
client = MongoClient(MONGO_CONNECTION_STRING)
db = client['kinconnect']
profiles_collection = db['profiles']

class CareerEntry(BaseModel):
    company: str
    title: str
    description: str
    start_date: str
    end_date: str

class ProjectEntry(BaseModel):
    title: str
    description: str

class ProfileModel(BaseModel):
    name: str = Field(..., title="Name of the person")
    honors: list[str] = Field(None, title="Honors, Awards and recognition they have recieved in life")
    interests: list[str] = Field(..., title="Interests and current focus of theirs the work or the event")
    skills: list[str] = Field(..., title="Skills they have")
    career: List[CareerEntry] = Field(..., title="Career history of the person")
    past_projects: List[ProjectEntry] = Field(..., title="Projects they have worked on")
    elevator_pitch: str = Field(..., title="Elevator pitch for the person for the event")

    class Config:
        arbitrary_types_allowed = True
zipped_profiles = zip(profile_details, career_profiles, project_profiles)


fake_profiles = [
    ProfileModel(
        name=profile_details.name,
        honors=profile_details.honors,
        interests=profile_details.interests,
        skills=profile_details.skills,
        career=[
            CareerEntry(
                company=career_profile['company'],
                title=career_profile['title'],
                description=career_profile['description'],
                start_date=career_profile['start_date'],
                end_date=career_profile['end_date']
            )
        ],
        past_projects=[
            ProjectEntry(
                title=project_profile['title'],
                description=project_profile['description']
            )
        ],
        elevator_pitch=profile_details.elevator_pitch
    )
    for profile_details, career_profile, project_profile in zipped_profiles
        
]

# Upsert fake profiles into MongoDB based on the name of the profile
for profile in fake_profiles:
    profiles_collection.update_one(
        {"name": profile.name},
        {"$set": profile.dict()},
        upsert=True
    )


In [14]:
fake_profiles[5]

ProfileModel(name='Sara Patel', honors=[], interests=['AI', 'critical thinking', 'problem solving', 'philosophical questions'], skills=['product management', 'AI', 'robotics', 'baking'], career=[CareerEntry(company='Google', title='Senior Software Engineer', description='Worked as a Senior Software Engineer at Google from January 2016 to present. During this time, I contributed to various projects and gained valuable experience in software development.', start_date='January 2016', end_date='Present')], past_projects=[ProjectEntry(title='AI-Powered Educational Bot for Critical Thinking', description='Developed an AI-powered bot designed to assist students in learning how to ask questions, think critically, and solve problems more effectively. The bot interacts with students through natural language processing (NLP) to guide them in exploring various topics, prompting them with questions that encourage deeper understanding and critical analysis. The project aimed to create a virtual tuto